In [156]:
import cv2
import mediapipe as mp
import numpy as np
from sklearn.cluster import KMeans
import urllib

def get_avg_color(region):
    avg_color_per_row = np.average(region, axis=0)
    avg_color = np.average(avg_color_per_row, axis=0)
    return avg_color

def bgr_to_hex(bgr):
    r, g, b = int(bgr[2]), int(bgr[1]), int(bgr[0])
    return "#{:02x}{:02x}{:02x}".format(r, g, b).upper()

def sample_feature_color(frame, landmarks, idx, region_size=6):
    h, w, _ = frame.shape
    x = int(landmarks.landmark[idx].x * w)
    y = int(landmarks.landmark[idx].y * h)
    patch = frame[y-region_size:y+region_size, x-region_size:x+region_size]
    if patch.size == 0:
        return None, (x, y)
    avg_bgr = get_avg_color(patch)
    return avg_bgr, (x, y)


# Initialize mediapipe face mesh
mp_face_mesh = mp.solutions.face_mesh
face_mesh = mp_face_mesh.FaceMesh(max_num_faces=1, refine_landmarks=True, min_detection_confidence=0.5, min_tracking_confidence=0.5)
def extract_features(path):
    values = []
    
    req = urllib.request.urlopen(path)
    arr = np.asarray(bytearray(req.read()), dtype=np.uint8)
    image = cv2.imdecode(arr, -1) # 'Load it as it is'
    
    rgb_image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    results = face_mesh.process(rgb_image)
    landmarks = results.multi_face_landmarks[0]
    
    features = {
        'Eye': 469,   # Left iris center
        'Lips': 13,   # Lower lip center
        'Cheek': 205, # Left cheek
        'Hair': 295   # Eyebrow, in place for hair color
    }
    feature_colors = {}
    
    for i, (feature, idx) in enumerate(features.items()):
        color_bgr, (x, y) = sample_feature_color(image, landmarks, idx)
        if color_bgr is not None:
            feature_colors[feature] = color_bgr
            r, g, b = int(color_bgr[2]), int(color_bgr[1]), int(color_bgr[0])
            hex_color = bgr_to_hex(color_bgr)
                           
            # Text display
            text = f"{feature}: RGB({r},{g},{b}) | {hex_color}"
            values += [r, g, b, hex_color]
            
    return values

In [161]:
import pandas as pd

celebs = pd.read_csv("celebs.csv").transform(lambda s: s.apply(lambda d: d.strip()))

celeb_features = pd.DataFrame(columns=["Eye_R", "Eye_G", "Eye_B", "Eye_Hex", "Lips_R", "Lips_G", "Lips_B", "Lips_Hex", "Cheek_R", "Cheek_G", "Cheek_B", "Cheek_Hex", "Hair_R", "Hair_G", "Hair_B", "Hair_Hex"])
for path in celebs["image"]:
    try:
        celeb_features.loc[celeb_features.shape[0]] = extract_features(r"{}".format(celebs["image"][0]))
    except: celeb_features.loc[celeb_features.shape[0]] = [None] * 16

pd.concat([celebs, celeb_features], axis=1)

,name,season,undertone,image,Eye_R,Eye_G,Eye_B,Eye_Hex,Lips_R,Lips_G,Lips_B,Lips_Hex,Cheek_R,Cheek_G,Cheek_B,Cheek_Hex,Hair_R,Hair_G,Hair_B,Hair_Hex
0,Beyonce,Autumn,Warm,https://static.wikia.nocookie.net/beyonce/imag...,93,101,108,#5D656C,127,60,55,#7F3C37,138,78,60,#8A4E3C,77,50,41,#4D3229
1,Lady Gaga,Autumn,Warm,https://hips.hearstapps.com/hmg-prod/images/la...,109,118,125,#6D767D,122,56,52,#7A3834,138,78,60,#8A4E3C,89,60,51,#593C33
2,Selena Gomez,Winter,Neutral,https://upload.wikimedia.org/wikipedia/commons...,114,124,130,#727C82,84,31,26,#541F1A,140,80,61,#8C503D,85,56,46,#55382E
3,Hrithik Roshan,Autumn,Warm,https://i.pinimg.com/originals/9c/d8/41/9cd841...,112,120,126,#70787E,99,41,36,#632924,139,78,60,#8B4E3C,80,52,42,#50342A
4,Sarah Rafferty,Autumn,Neutral,https://i.pinimg.com/564x/8a/54/0d/8a540d9394e...,110,119,125,#6E777D,118,54,49,#763631,140,80,62,#8C503E,83,55,45,#53372D
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,Prince Harry,Spring,Warm,https://images.hellomagazine.com/horizon/squar...,112,120,126,#70787E,87,33,28,#57211C,140,80,61,#8C503D,83,55,43,#53372B
96,Michael Peña,Autumn,Warm,https://upload.wikimedia.org/wikipedia/commons...,97,104,111,#61686F,75,25,21,#4B1915,139,78,60,#8B4E3C,80,53,42,#50352A
97,Taylor Zakhar Perez,Winter,Cool,https://m.media-amazon.com/images/M/MV5BMjNhZT...,100,107,113,#646B71,114,51,48,#723330,140,80,61,#8C503D,83,55,46,#53372E
98,Maisie Richardson-Sellers,Winter,Cool,https://m.media-amazon.com/images/M/MV5BZWZiYW...,108,117,124,#6C757C,102,42,38,#662A26,139,79,60,#8B4F3C,77,50,41,#4D3229


In [168]:
pd.concat([celebs, celeb_features], axis=1).drop("image", axis=1).to_csv("training_celebs.csv", index=False)

In [172]:
skin = [14, 0.39, 0.39] #HSL

analogous = [[skin[0] - 10, skin[0] + 10], skin[1], skin[2]]
complementary = [[[skin[0] + 170, skin[0] + 190]], skin[1], skin[2]]
options = {
    "winter": ,
    "spring": ,
    "summer": ,
    "autumn": 
}

def generate_clothing_colors(skin, season):
    # Step 1: Normalize and convert RGB to HSL
    r, g, b = [x / 255.0 for x in skin_rgb]
    h, l, s = colorsys.rgb_to_hls(r, g, b)  # Hue, Lightness, Saturation

    # Step 2: Generate harmonious hues (in degrees)
    hue_deg = h * 360
    analogous = [(hue_deg + 30) % 360, (hue_deg - 30) % 360]
    complementary = [(hue_deg + 180) % 360]
    triadic = [(hue_deg + 120) % 360, (hue_deg + 240) % 360]

    # Step 3: Convert back to RGB with good saturation/lightness values
    result_colors = []
    all_hues = analogous + complementary + triadic

    for hue in all_hues:
        h_norm = hue / 360.0
        # Pick a range of lightness & saturation to vary results
        for _ in range(2):  # Two shades per harmony
            s_val = random.uniform(0.5, 0.9)
            l_val = random.uniform(0.35, 0.65)
            r_new, g_new, b_new = colorsys.hls_to_rgb(h_norm, l_val, s_val)
            result_colors.append((
                int(r_new * 255),
                int(g_new * 255),
                int(b_new * 255)
            ))

    return result_colors

In [ ]:
import colorsys
import random

def generate_clothing_colors(skin_rgb, season):
    # Step 1: Normalize and convert RGB to HSL
    r, g, b = [x / 255.0 for x in skin_rgb]
    h, l, s = colorsys.rgb_to_hls(r, g, b)  # Hue, Lightness, Saturation

    hue_deg = h * 360
    analogous = [(hue_deg + 30) % 360, (hue_deg - 30) % 360]
    complementary = [(hue_deg + 150) % 360, (hue_deg + 180) % 360, (hue_deg + 210) % 360]
    triadic = [(hue_deg + 120) % 360, (hue_deg + 240) % 360]

    all_hues = analogous + complementary + triadic

    options = {
        "summer": (0.55, 0.75),
        "autumn": (0.95, 0.35),
        "winter": (0.90, 0.40),
        "spring": (0.85, 0.50)
    }
    
    results = [colorsys.hls_to_rgb(hue/360, options[season][1], options[season][0]) for hue in all_hues]
    return np.array(results) * 255

generate_clothing_colors([138, 79, 61], 'summer')

[44.02597402597402, 344.02597402597405, 164.02597402597402, 194.02597402597402, 224.02597402597402, 134.02597402597402, 254.02597402597402]


array([[226.3125    , 207.64285714, 156.1875    ],
       [226.3125    , 156.1875    , 174.85714286],
       [156.1875    , 226.3125    , 207.64285714],
       [156.1875    , 209.91964286, 226.3125    ],
       [156.1875    , 174.85714286, 226.3125    ],
       [156.1875    , 226.3125    , 172.58035714],
       [172.58035714, 156.1875    , 226.3125    ]])